In [55]:
import pandas as pd
from dotenv import load_dotenv
import os
from lyricsgenius import Genius
import pickle
import numpy as np

In [56]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
genius = Genius(API_KEY)
genius.remove_section_headers = True


In [57]:
df = pd.read_csv('Hot 100.csv')

In [58]:
df['chart_date'] = pd.to_datetime(df.chart_date)
df['chart_debut'] = pd.to_datetime(df.chart_debut)

In [59]:
df['consecutive_weeks'].fillna(value=0,inplace=True)
df['previous_week'].fillna(value=0,inplace=True)

In [60]:
frame = pd.DataFrame(df.dtypes)
numericals = list(frame.loc[frame.iloc[:,0]=='float64'].iloc[:,0].keys())
numericals = numericals + list(frame.loc[frame.iloc[:,0]=='int64'].iloc[:,0].keys())
for x in numericals:
    df[x] = df[x].astype('int32')

In [61]:
df.sort_values(by='chart_date',inplace=True)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335195 entries, 100546 to 79691
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   chart_position     335195 non-null  int32         
 1   chart_date         335195 non-null  datetime64[ns]
 2   song               335195 non-null  object        
 3   performer          335195 non-null  object        
 4   song_id            335195 non-null  object        
 5   instance           335195 non-null  int32         
 6   time_on_chart      335195 non-null  int32         
 7   consecutive_weeks  335195 non-null  int32         
 8   previous_week      335195 non-null  int32         
 9   peak_position      335195 non-null  int32         
 10  worst_position     335195 non-null  int32         
 11  chart_debut        335195 non-null  datetime64[ns]
 12  chart_url          335195 non-null  object        
dtypes: datetime64[ns](2), int32(7), object(4

In [63]:

unique_df = df.drop_duplicates(subset=['song_id'])

In [64]:
unique_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30314 entries, 100546 to 305276
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   chart_position     30314 non-null  int32         
 1   chart_date         30314 non-null  datetime64[ns]
 2   song               30314 non-null  object        
 3   performer          30314 non-null  object        
 4   song_id            30314 non-null  object        
 5   instance           30314 non-null  int32         
 6   time_on_chart      30314 non-null  int32         
 7   consecutive_weeks  30314 non-null  int32         
 8   previous_week      30314 non-null  int32         
 9   peak_position      30314 non-null  int32         
 10  worst_position     30314 non-null  int32         
 11  chart_debut        30314 non-null  datetime64[ns]
 12  chart_url          30314 non-null  object        
dtypes: datetime64[ns](2), int32(7), object(4)
memory usage:

In [65]:
def get_lyrics_genius(row):
    ans = float('nan')
    title = row['song']
    artist = row['performer']
    try:
        song = genius.search_song(title, artist=artist)
    except:
        ans = float('nan')
    try:
        song.lyrics
    except:
        ans = float('nan')
    else:
        if song.lyrics[:len(title)]==title:
            ans = song.lyrics[len(title)+len(' lyrics'):]
        else:
            ans = float('nan')
    return(ans)


In [66]:
start = 1
stop = 2
subset_df = unique_df.iloc[start:stop]

In [67]:
subset_df['lyrics'] = subset_df.apply(get_lyrics_genius,axis=1)

Searching for "Fiesta" by R. Kelly Featuring Jay-Z...
Done.
Searching for "Superwoman Pt. II" by Lil' Mo Featuring Fabolous...
Done.
Searching for "Right Where I Need To Be" by Gary Allan...
Done.
Searching for "There It Is" by Ginuwine...
Done.
Searching for "Too Little Too Late" by Barenaked Ladies...
Done.
Searching for "Play" by Jennifer Lopez...
Done.
Searching for "Guilty Until Proven Innocent" by Jay-Z Featuring R. Kelly...
Done.
Searching for "La Bomba" by Azul Azul...
Done.
Searching for "I Like Them Girls" by Tyrese...
Done.
Searching for "She Couldn't Change Me" by Montgomery Gentry...
Done.
Searching for "It's Been Awhile" by Staind...
Done.
Searching for "Puppy Love" by Lil Bow Wow Featuring Jagged Edge...
Done.
Searching for "If You Can Do Anything Else" by George Strait...
Done.
Searching for "Lady Marmalade" by Christina Aguilera, Lil' Kim, Mya & P!nk...
Done.
Searching for "Peaches & Cream" by 112...
Done.
Searching for "Until The End Of Time" by 2Pac...
Done.
Searchin

C:\Users\kapta\AppData\Local\Temp\ipykernel_4668\3462107902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['lyrics'] = subset_df.apply(get_lyrics_genius,axis=1)


In [68]:
subset_df.to_pickle('./dataframe'+str(start)+'-'+str(stop)+'.pkl')

In [70]:
first_pass_df = pd.read_pickle('./complete_df.pkl')

In [72]:
missing_df = first_pass_df.loc[first_pass_df.lyrics.isna()]

In [73]:
unique_df['og_performer'] = unique_df['performer']
unique_df['og_song'] = unique_df['song']

C:\Users\kapta\AppData\Local\Temp\ipykernel_4668\1350453345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['og_performer'] = unique_df['performer']
C:\Users\kapta\AppData\Local\Temp\ipykernel_4668\1350453345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['og_song'] = unique_df['song']


In [74]:
unique_df

,chart_position,chart_date,song,performer,song_id,instance,time_on_chart,consecutive_weeks,previous_week,peak_position,worst_position,chart_debut,chart_url,og_performer,og_song
100546,74,1958-08-02,Gotta Have Rain,Eydie Gorme,Gotta Have RainEydie Gorme,1,1,0,0,74,74,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,Eydie Gorme,Gotta Have Rain
274332,79,1958-08-02,The Bird On My Head,David Seville,The Bird On My HeadDavid Seville,1,1,0,0,79,79,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,David Seville,The Bird On My Head
30153,78,1958-08-02,Betty Lou Got A New Pair Of Shoes,Bobby Freeman,Betty Lou Got A New Pair Of ShoesBobby Freeman,1,1,0,0,78,78,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,Bobby Freeman,Betty Lou Got A New Pair Of Shoes
83086,10,1958-08-02,Fever,Peggy Lee,FeverPeggy Lee,1,1,0,0,10,10,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,Peggy Lee,Fever
309814,23,1958-08-02,Western Movies,The Olympics,Western MoviesThe Olympics,1,1,0,0,23,23,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,The Olympics,Western Movies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169760,99,2022-10-22,Lokera,"Rauw Alejandro, Lyanno & Brray","LokeraRauw Alejandro, Lyanno & Brray",1,1,0,0,99,99,2022-10-22,https://www.billboard.com/charts/hot-100/2022-...,"Rauw Alejandro, Lyanno & Brray",Lokera
292528,83,2022-10-22,To The Bone,"Quavo, Takeoff & YoungBoy Never Broke Again","To The BoneQuavo, Takeoff & YoungBoy Never Bro...",1,1,0,0,83,83,2022-10-22,https://www.billboard.com/charts/hot-100/2022-...,"Quavo, Takeoff & YoungBoy Never Broke Again",To The Bone
216996,65,2022-10-22,Poland,Lil Yachty,PolandLil Yachty,1,1,0,0,65,65,2022-10-22,https://www.billboard.com/charts/hot-100/2022-...,Lil Yachty,Poland
111292,77,2022-10-22,Heyy,Lil Baby,HeyyLil Baby,1,1,0,0,77,77,2022-10-22,https://www.billboard.com/charts/hot-100/2022-...,Lil Baby,Heyy


In [75]:
missing_df.merge(unique_df,how='left',on='song_id')

,chart_position_x,chart_date_x,song_x,performer_x,song_id,instance_x,time_on_chart_x,consecutive_weeks_x,previous_week_x,peak_position_x,...,instance_y,time_on_chart_y,consecutive_weeks_y,previous_week_y,peak_position_y,worst_position_y,chart_debut_y,chart_url_y,og_performer,og_song
0,78,1958-08-02,Betty-Lou-Got-A-New-Pair-Of-Shoes,Bobby-Freeman,Betty Lou Got A New Pair Of ShoesBobby Freeman,1,1,0,0,78,...,1,1,0,0,78,78,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,Bobby Freeman,Betty Lou Got A New Pair Of Shoes
1,54,1958-08-02,Nel-Blu-Dipinto-Di-Blu-(Volaré),Domenico-Modugno,Nel Blu Dipinto Di Blu (Volaré)Domenico Modugno,1,1,0,0,54,...,1,1,0,0,54,54,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,Domenico Modugno,Nel Blu Dipinto Di Blu (Volaré)
2,49,1958-08-02,All-I-Have-To-Do-Is-Dream,The-Everly-Brothers,All I Have To Do Is DreamThe Everly Brothers,1,1,0,0,49,...,1,1,0,0,49,49,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,The Everly Brothers,All I Have To Do Is Dream
3,21,1958-08-02,Guess-Things-Happen-That-Way,Johnny-Cash-And-The-Tennessee-Two,Guess Things Happen That WayJohnny Cash And Th...,1,1,0,0,21,...,1,1,0,0,21,21,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,Johnny Cash And The Tennessee Two,Guess Things Happen That Way
4,55,1958-08-02,Let's-Go-Steady-For-The-Summer,The-Three-G's,Let's Go Steady For The SummerThe Three G's,1,1,0,0,55,...,1,1,0,0,55,55,1958-08-02,https://www.billboard.com/charts/hot-100/1958-...,The Three G's,Let's Go Steady For The Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9439,32,2022-10-08,Star-Walkin'-(League-Of-Legends-Worlds-Anthem),Lil-Nas-X,Star Walkin' (League Of Legends Worlds Anthem)...,1,1,0,0,32,...,1,1,0,0,32,32,2022-10-08,https://www.billboard.com/charts/hot-100/2022-...,Lil Nas X,Star Walkin' (League Of Legends Worlds Anthem)
9440,100,2022-10-08,Down-Home,Jimmie-Allen,Down HomeJimmie Allen,1,1,0,0,100,...,1,1,0,0,100,100,2022-10-08,https://www.billboard.com/charts/hot-100/2022-...,Jimmie Allen,Down Home
9441,95,2022-10-08,Out-In-The-Middle,Zac-Brown-Band,Out In The MiddleZac Brown Band,1,1,0,0,95,...,1,1,0,0,95,95,2022-10-08,https://www.billboard.com/charts/hot-100/2022-...,Zac Brown Band,Out In The Middle
9442,99,2022-10-15,What-He-Didn't-Do,Carly-Pearce,What He Didn't DoCarly Pearce,1,1,0,0,99,...,1,1,0,0,99,99,2022-10-15,https://www.billboard.com/charts/hot-100/2022-...,Carly Pearce,What He Didn't Do
